### Whisper STT model without fine-tuning

In [16]:
from datasets import load_dataset, DatasetDict, Audio

dataset = DatasetDict()

dataset = load_dataset("data_gtts", data_dir="I:/Repos/STT_FineTune/nats/data_gtts")

# Resample to 16kHz
dataset = dataset.cast_column('audio', Audio(sampling_rate=16000))

dataset

Using custom data configuration default-9b41ac5bfd8c70c2
Found cached dataset data_gtts (I:/Repos/HFdatasets/data_gtts/default-9b41ac5bfd8c70c2/0.1.0/99611922a2fe30672e990db44b070dc747a16dd2cb691d0d2c33dc670a2e3b68)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 10
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 5
    })
})

In [36]:
# Load pre-trained model
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import IPython.display as ipd
import numpy as np
import random

idx = np.random.randint(0, len(dataset['test']))
display(ipd.Audio(data=np.asarray(dataset["test"][idx]["audio"]["array"]), autoplay=False, rate=16000))
print(dataset['test'][idx]['transcription'])

# Whisper
hf_model = "openai/whisper-tiny.en"
processor = WhisperProcessor.from_pretrained(hf_model)
model = WhisperForConditionalGeneration.from_pretrained(hf_model)
input_features = processor(dataset['test'][idx]["audio"]["array"], return_tensors="pt", sampling_rate=16000).input_features 
generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Whisper: " + transcription)

hf_model = "facebook/wav2vec2-large-960h-lv60-self"
processor = Wav2Vec2Processor.from_pretrained(hf_model)
model = Wav2Vec2ForCTC.from_pretrained(hf_model)
input_values = processor(dataset["test"][idx]["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate=16000).input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print("Wav2Vec2: " + transcription[0])

Speedbird One Golf November Mach decimal Eight Zero
Whisper:  Speedbird 1 Golf November Muck Dec. 8–0


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2: SPEEDBIRD ONE GOLF NOVEMBER MOCK DECIMAL EIGHT ZERO
